Redes Neuronales: se desea realizar una predicción de ph por medio de redes neuronales, por la libreria Keras de tensorflow

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import BatchNormalization

In [6]:
df = pd.read_csv('pH Data.csv')

dataset = df.values
X = dataset[:,:-1]
y = dataset[:,-1]


In [7]:
scale = MinMaxScaler()
X_scale = scale.fit_transform(X)
X_tr, X_ts, y_tr, y_ts = train_test_split(X_scale, y, test_size=0.2, random_state=7)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

model_ = Sequential([
                     Flatten(input_shape=(3,)),
                     Dense(100, activation='relu'),
                     BatchNormalization(),
                     Dense(100, activation='relu'),
                     BatchNormalization(),
                     Dense(45, activation='relu'),
                     BatchNormalization(),
                     Dense(15, activation='softmax'),
                    ])
#adam_optimizer = Adam(learning_rate=0.0001)

model_.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2022-10-04 03:39:51.998865: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-04 03:39:51.998903: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-04 03:39:51.998927: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (yordan-H81M-H): /proc/driver/nvidia/version does not exist
2022-10-04 03:39:52.094727: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# modelCheckpoint = ModelCheckpoint('best_banknote_model.hdf5', save_best_only = True)  --> for model checkpoint
early_stopping_monitor = EarlyStopping(patience=2, monitor='val_acc',)
hist = model_.fit(X_tr, y_tr, epochs=30, validation_split=0.15, callbacks=[early_stopping_monitor], batch_size=10) # callbacks = [monitor_val_acc, modelCheckpoint]--> for model checkpoint

Epoch 1/30
45/45 [==============================] - 5s 25ms/step - loss: 2.2112 - accuracy: 0.3454 - val_loss: 2.6350 - val_accuracy: 0.2405
Epoch 2/30
45/45 [==============================] - 1s 15ms/step - loss: 1.5282 - accuracy: 0.5147 - val_loss: 2.5840 - val_accuracy: 0.0886
Epoch 3/30
45/45 [==============================] - 1s 12ms/step - loss: 1.4095 - accuracy: 0.5237 - val_loss: 2.5258 - val_accuracy: 0.1013
Epoch 4/30
45/45 [==============================] - 1s 19ms/step - loss: 1.3689 - accuracy: 0.5418 - val_loss: 2.4675 - val_accuracy: 0.1519
Epoch 5/30
45/45 [==============================] - 1s 18ms/step - loss: 1.3278 - accuracy: 0.5440 - val_loss: 2.3700 - val_accuracy: 0.2278
Epoch 6/30
45/45 [==============================] - 1s 14ms/step - loss: 1.2595 - accuracy: 0.6140 - val_loss: 2.2008 - val_accuracy: 0.3165
Epoch 7/30
45/45 [==============================] - 1s 16ms/step - loss: 1.2837 - accuracy: 0.5530 - val_loss: 1.9917 - val_accuracy: 0.3924
Epoch 8/30
45

In [10]:
model_.evaluate(X_tr,y_tr)

17/17 [==============================] - 0s 9ms/step - loss: 0.8048 - accuracy: 0.7548


[0.8047838807106018, 0.7547892928123474]

In [11]:
print(model_.summary())
print('\n')
print(model_.layers)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3)                 0         
                                                                 
 dense (Dense)               (None, 100)               400       
                                                                 
 batch_normalization (BatchN  (None, 100)              400       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 batch_normalization_1 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 45)                4

In [12]:
y_pred = np.argmax(model_.predict(X_ts), axis=-1)
print('predicted values for X_ts= \n', y_pred)

zeros = y_pred - y_ts
c = 0
for i in zeros:
  if i == 0:
    c = c + 1
acc = c / len(zeros)   
print('\n') 
print('accuracy_score: ',acc)

5/5 [==============================] - 0s 2ms/step
predicted values for X_ts= 
 [ 7  9  7 13  0  3  4 12  6  9  8 10  4 10  6  8 12 13  9  3  9  1  1  2
  6 12 13  3 14  6  4  7  0  8  1 10 10  2 11  4  5  1  5  9 10  5  0 13
  2  0  0 10  4 11  4  5  3  3 10  6  3 10  7  8  8  9 14 13  7  1  0  1
 14  1 10 13  3 13  8 14  1  2  5 11  9  2  3  1  1 10  7  2  0  6  0  7
  5  4 14  7 13 10 12 13 10  7  0 12  2 14 14  5 10 14  1 14  3  0  4 14
  9  3  1 12  6  7  2 11 14  5 14]


accuracy_score:  0.6946564885496184
